# ARMA(2,1) パラメータ推定
ARMA(2,1)のデータを生成して，パラメータ推定の様子を見る

statsmodels.tsa.arima_process.ArmaProcess  
http://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_process.ArmaProcess.html  

statsmodels.tsa.arima_process.ArmaProcess.generate_sample  
http://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_process.ArmaProcess.generate_sample.html  

statsmodels.tsa.arima_model.ARMA  
http://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARMA.html

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
  
from statsmodels.tsa.arima_process import arma_generate_sample
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima_model import ARMA

np.random.seed(123)
FLAG_fig = False

データ生成 arma_generate_sample<br>
引数の説明：  
distrvs: ARMAへの入力でディフォルトは標準正規分布(0, 1) as np.random.randn
burnin: 初期期間を何点かで定め，この期間後にデータを出力する。過渡現象を観測しないため。

In [ ]:
nobs = 10000 #データ数
ar = [1, -1.5, 0.7] # pole = 0.75 +/- 0.37 j < unit circle
ma = [1.0, 0.6]
dist = lambda n: np.random.randn(n)  # 正規分布 N(0,1),  引数 n はダミー
#dist = lambda n: np.random.standard_t(3, size=n)  # t 分布
x = arma_generate_sample(ar, ma, nobs, sigma=1, distrvs=dist, burnin=500)
print(type(x))

下記の関数は次を参照<br>

statsmodels.tsa.arima_model.ARMA<br>
http://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARMA.html<br>
.fit()の作用　statsmodels.tsa.arima_model.ARMA.fit<br>
http://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARMA.fit.html<br>
.fit()の結果　statsmodels.tsa.arima_model.ARMAResults<br>
http://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARMAResults.html
    

真の次数は未知として，幾つかの推定次数を用い，比較する<br>
もし，"FutureWarning"というWarningは無視して構わない（2018年現在）

In [ ]:
arma20 = ARMA(x, order=(2,0)).fit( trend='nc' ) # AR model
arma21 = ARMA(x, order=(2,1)).fit( trend='nc' ) # trend: バイアスは無い
arma32 = ARMA(x, order=(3,2)).fit( trend='nc' )
arma43 = ARMA(x, order=(4,3)).fit( trend='nc' )

## 注意！
ARのパラメータの符号は反転している。これは，モデルの立て方でAR部の符号反転を行っているためである。  
http://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_process.ArmaProcess.html  


このことは，次の例題でもARの符号を反転して与えていることからもわかる<br>
http://www.statsmodels.org/dev/examples/notebooks/generated/tsa_arma_1.html  

また，極は$z$でなく$z^{-1}$で見ており，単位円外ならば安定極となっている。

## 注意！
パッケージのバージョン更新に伴い，テキストや書籍の計算結果と少しずつ変わることがある。

In [ ]:
print('arma21-----------summary--------------------')
print(arma21.summary())

summary（）だけでは，モデル評価がよくわからないので，sigma2（誤差分散）とAICを見る

In [ ]:
print('arma20: sigma2 =',arma20.sigma2, '  AIC =',arma20.aic)
print('arma21: sigma2 =',arma21.sigma2, ' AIC =',arma21.aic)
print('arma32: sigma2 =',arma32.sigma2, ' AIC =',arma32.aic)
print('arma43: sigma2 =',arma43.sigma2, ' AIC =',arma43.aic)

In [ ]:
print('arma20 parameters = ',arma20.params)
print('arma21 parameters = ',arma21.params)
print('arma32 parameters = ',arma32.params)
print('arma43 parameters = ',arma43.params)

In [ ]:
fig = plt.subplots(figsize=(10,3))
plt.plot(x, label='x')
resid = arma21.resid  # short for residual
#print(len(resid))
plt.plot(resid, label='resid')
plt.xlabel('k')
plt.legend()

if FLAG_fig: plt.savefig('fig_ARMA_ParmEst_PlotResid.png', bbox_inches='tight')
plt.show()

#### 白色性の検定

In [ ]:
auto_corr = np.correlate(resid, resid, mode='full')
center = int(len(auto_corr)/2)
AutoR = auto_corr[center:]/np.max(auto_corr)
plt.xlabel('k')
plt.plot(AutoR)

if FLAG_fig: plt.savefig('fig_ARMA_ParmEst_AutoCorr.png')
plt.show()
    
count = 0
for k in np.arange(1,len(AutoR)-1):
    if np.abs(AutoR[k]) > 2.17/np.sqrt(len(AutoR)):
        count += 1
    if np.abs(AutoR[k]) < -2.17/np.sqrt(len(AutoR)):
        count += 1
#        print("Warning", i, AutoR[i])
print('count = ', count, ' len(AutoR) = ', len(AutoR), '  rate =', count/len(AutoR))
print('k >= 1, max(AutoR[k] =', np.max(AutoR[1:]),  '    min(AutorR[k] =', np.min(AutoR[1:]))